In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    
    if PARAM['atmpro'] == PARAM['atmpro']:
        atmpro = PARAM['atmpro']
    else:
        atmpro = 'mls'
        
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'co2': 0.0004}
band = [11]
atmpro = saw
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o co2 band9
  atmpro = mls
  band = 9
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.5e-24
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [3, 3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (50, 250)]
  tsfc = 294
  vmax = 3000
  vmin = 1900
  w_diffuse = [(1.66, 1.66, 1.75), (1.75, 1.6, 1.85)]
  wgt = [(0.7, 0.8, 0.7), (0.8, 0.7, 0.8)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.000964                    1  0.098340
0.000750        2 -0.155994                    2 -0.131755
0.001052        3 -0.112857                    3 -0.094402
0.001476        4 -0.091207                    4 -0.078541
0.002070        5 -0.074267                    5 -0.076646
0.002904        6 -0.057905                    6 -0.080242
0.004074        7 -0.041275                    7 -0.082445
0.005714        8 -0.024465                    8 -0.072168
0.008015        9 -0.007901                    9 -0.053307
0.011243       10  0.008007                   10 -0.026893
0.015771       11  0.023183                   11  0.005232
0.022122       12  0.038618                   12  0.041972
0.031031       13  0.059943                   13  0.086393
0.043528       14  0.075282                   14  0.121832
0.061057       15  0.072367                   15  0.132255
0.085645       16  0.067575                   16  0.127747
0.120136       17  0.069055                   17  0.115865
0.168516       18  0.073797                   18  0.098137
0.236378       19  0.082006                   19  0.080468
0.331549       20  0.096763                   20  0.075636
0.465100       21  0.132414                   21  0.106414
0.652400       22  0.119620                   22  0.096487
0.915100       23  0.061907                   23  0.050870
1.283650       24  0.030065                   24  0.034737
1.800600       25  0.016397                   25  0.027480
2.525700       26  0.008810                   26  0.019332
3.542800       27  0.003831                   27  0.010448
4.969550       28  0.001619                   28  0.003258
6.970850       29  0.001396                   29  0.000392
9.778100       30  0.001181                   30  0.000706
13.715850      31  0.000604                   31  0.001377
19.239350      32 -0.000120                   32  0.000878
26.987250      33 -0.000274                   33  0.000423
37.855300      34  0.000020                   34  0.000302
53.100050      35  0.000096                   35  0.000001
73.887500      36  0.000112                   36 -0.000122
97.662500      37  0.000127                   37 -0.000086
121.437500     38 -0.000019                   38 -0.000219
145.212500     39 -0.000209                   39 -0.000408
168.987500     40 -0.000322                   40 -0.000503
192.762500     41 -0.000424                   41 -0.000605
216.537500     42 -0.000530                   42 -0.000724
240.312500     43 -0.000652                   43 -0.000859
264.087500     44 -0.000840                   44 -0.001059
287.862500     45 -0.001261                   45 -0.001530
311.637500     46 -0.001548                   46 -0.001851
335.412500     47 -0.001367                   47 -0.001687
359.187500     48 -0.001124                   48 -0.001463
382.962500     49 -0.001020                   49 -0.001379
406.737500     50 -0.000930                   50 -0.001276
430.512500     51 -0.000831                   51 -0.001133
454.287500     52 -0.000715                   52 -0.000997
478.062500     53 -0.000578                   53 -0.000764
501.837500     54 -0.000419                   54 -0.000533
525.612500     55 -0.000236                   55 -0.000260
549.387500     56 -0.000032                   56  0.000046
573.162500     57  0.000190                   57  0.000379
596.937500     58  0.000421                   58  0.000738
620.712500     59  0.000650                   59  0.001024
644.487500     60  0.000866                   60  0.001401
668.262500     61  0.001063                   61  0.001673
692.037500     62  0.001238                   62  0.001928
715.812500     63  0.001395                   63  0.002164
739.587500     64  0.001547                   64  0.002396
763.362500     65  0.001709                   65  0.002642
78

# Fluxes by Layer

CRD                        CLIRAD                \
                       flug      fldg     fnetg      flug          fldg   
pressure    level                                                         
0.000000    1     -1.267380  0.000000 -1.267380 -1.235431  8.379454e-09   
0.000624    2     -1.267395  0.000016 -1.267380 -1.235454  3.013485e-05   
0.000876    3     -1.267403  0.000019 -1.267384 -1.235466  3.765633e-05   
0.001229    4     -1.267414  0.000025 -1.267389 -1.235481  4.873732e-05   
0.001723    5     -1.267429  0.000034 -1.267394 -1.235500  6.380328e-05   
0.002417    6     -1.267448  0.000048 -1.267400 -1.235526  8.306878e-05   
0.003391    7     -1.267474  0.000066 -1.267407 -1.235559  1.068801e-04   
0.004757    8     -1.267507  0.000093 -1.267414 -1.235602  1.365637e-04   
0.006672    9     -1.267551  0.000131 -1.267419 -1.235657  1.756095e-04   
0.009359    10    -1.267607  0.000185 -1.267422 -1.235728  2.292640e-04   
0.013128    11    -1.267678  0.000260 -1.267418 -1.235817  3.060259e-04   
0.018415    12    -1.267769  0.000365 -1.267404 -1.235927  4.191098e-04   
0.025830    13    -1.267879  0.000509 -1.267370 -1.236059  5.884762e-04   
0.036232    14    -1.268002  0.000706 -1.267296 -1.236207  8.428550e-04   
0.050823    15    -1.268126  0.000960 -1.267166 -1.236361  1.207887e-03   
0.071291    16    -1.268256  0.001266 -1.266990 -1.236530  1.696716e-03   
0.100000    17    -1.268401  0.001640 -1.266760 -1.236719  2.321013e-03   
0.140271    18    -1.268552  0.002121 -1.266431 -1.236925  3.079765e-03   
0.196760    19    -1.268691  0.002753 -1.265937 -1.237132  3.943371e-03   
0.275997    20    -1.268767  0.003599 -1.265168 -1.237307  4.873843e-03   
0.387100    21    -1.268644  0.004750 -1.263894 -1.237322  5.884498e-03   
0.543100    22    -1.267742  0.006295 -1.261447 -1.236513  7.042180e-03   
0.761700    23    -1.265939  0.007590 -1.258350 -1.234696  7.723464e-03   
1.068500    24    -1.264056  0.007956 -1.256100 -1.232786  7.662818e-03   
1.498800    25    -1.262389  0.007822 -1.254567 -1.231079  7.726635e-03   
2.102400    26    -1.260919  0.007524 -1.253395 -1.229523  8.134909e-03   
2.949000    27    -1.259644  0.007133 -1.252511 -1.228108  8.659303e-03   
4.136600    28    -1.258647  0.006674 -1.251972 -1.226917  8.938319e-03   
5.802500    29    -1.257933  0.006280 -1.251653 -1.225996  8.660670e-03   
8.139200    30    -1.257358  0.006091 -1.251267 -1.225195  7.968050e-03   
11.417000   31    -1.256854  0.006046 -1.250808 -1.224438  7.485315e-03   
16.014700   32    -1.256505  0.006026 -1.250479 -1.223866  7.662838e-03   
22.464000   33    -1.256601  0.006030 -1.250571 -1.223883  8.351622e-03   
31.510500   34    -1.257282  0.006417 -1.250865 -1.224630  9.552039e-03   
44.200100   35    -1.258340  0.007506 -1.250834 -1.225825  1.120139e-02   
62.000000   36    -1.259756  0.009124 -1.250632 -1.227445  1.282401e-02   
85.775000   37    -1.261431  0.011115 -1.250316 -1.229359  1.439417e-02   
109.550000  38    -1.262856  0.012898 -1.249958 -1.231021  1.581511e-02   
133.325000  39    -1.264204  0.014192 -1.250011 -1.232637  1.681443e-02   
157.100000  40    -1.265712  0.015111 -1.250601 -1.234534  1.756185e-02   
180.875000  41    -1.267403  0.015896 -1.251507 -1.236737  1.834914e-02   
204.650000  42    -1.269294  0.016592 -1.252703 -1.239266  1.917603e-02   
228.425000  43    -1.271416  0.017220 -1.254196 -1.242163  2.003524e-02   
252.200000  44    -1.273827  0.017796 -1.256032 -1.245461  2.091406e-02   
275.975000  45    -1.276731  0.018334 -1.258397 -1.249339  2.180905e-02   
299.750000  46    -1.280917  0.018969 -1.261947 -1.254697  2.285905e-02   
323.525000  47    -1.286861  0.020553 -1.266308 -1.262071  2.502032e-02   
347.300000  48    -1.293996  0.023839 -1.270157 -1.270838  2.903610e-02   
371.075000  49    -1.301819  0.028496 -1.273323 -1.280390  3.446785e-02   
394.850000  50    -1.310265  0.034070 -1.276196 -1.290637  4.083125e-02   
418.625000  51    -1.319311  0.040495 -1.278816 -

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')